<a href="https://colab.research.google.com/github/pandreof/GooAluraNoChurn/blob/main/chatBot_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q -U google-generativeai
!pip install -q -U simple-salesforce

import google.generativeai as genai
from datetime import date, timedelta
from simple_salesforce import Salesforce

GOOGLE_API_KEY = ""
SFDC_INTEGRATION_USER= "ptandreo-lgxk@force.com"
SFDC_INTEGRATION_PASS= "@1Candidato"
SFDC_INTEGRATION_TOKEN= "xUvUe9D6j7PDAC05sYYtfuFIf" #Instância nova criada em Salesforce, sem dados sensíveis e expira em 30 dias
#cpf = "972.237.264-58" #número fake de cpf para teste com dados e probabilidade de churn
genai.configure(api_key=GOOGLE_API_KEY)

lgeneration_config = {
    "candidate_count" : 1,
    "temperature" : 0,
}

lsafety_settings = {
    "HARASSMENT" : "BLOCK_NONE",
    "HATE" : "BLOCK_NONE",
    "SEXUAL" : "BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE",
}

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=lgeneration_config,
                              safety_settings=lsafety_settings)




In [2]:
from simple_salesforce import Salesforce
from datetime import date, timedelta
import pandas as pd

class ChurnCalculator:
    @staticmethod
    def buscar_dados_cliente(cpf, username, password, security_token, domain='login'):
        """
        Busca dados do cliente no Salesforce, incluindo Cases, data do primeiro contrato
        e média das faturas.

        Args:
            cpf: CPF do cliente (string).
            username: Nome de usuário do Salesforce.
            password: Senha do Salesforce.
            security_token: Token de segurança do Salesforce.
            domain: Domínio do Salesforce (opcional, default 'login').

        Returns:
            Tupla contendo: lista de Cases, data do primeiro contrato (datetime) e média das faturas (float).
        """
        sf = Salesforce(username=username, password=password, security_token=security_token, domain=domain)

        # Calcula a data de 30 dias atrás
        data_limite = date.today() - timedelta(days=30)
        data_limite_str = data_limite.strftime('%Y-%m-%d')

        # Busca Cases
        query_cases = f"SELECT Id, CaseNumber, Subject, Description FROM Case WHERE Account.CPF__c = '{cpf}' AND Status = 'Escalated' AND CreatedDate >= {data_limite_str}T00:00:00Z"
        cases = sf.query_all(query_cases)['records']

        # Busca data do primeiro contrato
        query_cliente = f"SELECT Name, cliente_desde__c FROM Account WHERE CPF__c = '{cpf}'"
        #query_cliente = f"SELECT Name, cliente_desde__c FROM Account WHERE id = '001Kj00002QzrDHIAZ'"

        cliente = sf.query_all(query_cliente)['records']
        data_primeiro_contrato = None
        if cliente:
            data_primeiro_contrato = pd.to_datetime(cliente[0]['cliente_desde__c'])
            if data_primeiro_contrato is not None:
                data_primeiro_contrato = data_primeiro_contrato.to_pydatetime().date()
            tempo_relacionamento_meses = (date.today() - data_primeiro_contrato).days / 30 if data_primeiro_contrato is not None else 0

        # Busca média das faturas
        query_faturas = f"SELECT Amount FROM Payment WHERE Account.CPF__c = '{cpf}'"
        faturas = sf.query_all(query_faturas)['records']
        media_faturas = pd.DataFrame(faturas)['Amount'].mean() if faturas else 0

        return cliente, cases, data_primeiro_contrato, media_faturas

    @staticmethod
    def calcular_risco_churn(cases, data_primeiro_contrato, media_faturas):
        """
        Calcula o risco de churn usando média ponderada dos fatores:
        quantidade de Cases, tempo de relacionamento e média das faturas.

        Args:
            cases: Lista de Cases (chamados) do cliente.
            data_primeiro_contrato: Data do primeiro contrato (datetime).
            media_faturas: Média das faturas (float).

        Returns:
            Porcentagem de risco de churn (float).
        """
        # Calcula tempo de relacionamento em meses
        tempo_relacionamento_meses = (date.today() - data_primeiro_contrato).days / 30 if data_primeiro_contrato else 0

        # Define pesos para cada fator (ajuste conforme necessário)
        peso_cases = 0.5
        peso_tempo_relacionamento = 0.3
        peso_media_faturas = 0.2

        # Calcula os scores para cada fator (ajuste as faixas conforme necessário)
        score_cases = len(cases) * 10
        score_tempo_relacionamento = max(0, 10 - tempo_relacionamento_meses) # Quanto mais tempo, menor o score
        score_media_faturas = media_faturas / 100  # Score aumenta com a média das faturas

        # Calcula a média ponderada
        risco_churn = (
            (score_cases * peso_cases) +
            (score_tempo_relacionamento * peso_tempo_relacionamento) +
            (score_media_faturas * peso_media_faturas)
        )

        return risco_churn

In [6]:
cpf = input("Digite seu CPF: ")

# Obtém os dados do cliente
cliente, cases, data_primeiro_contrato, media_faturas = ChurnCalculator.buscar_dados_cliente(cpf, SFDC_INTEGRATION_USER, SFDC_INTEGRATION_PASS, SFDC_INTEGRATION_TOKEN)

if cliente is not None and len(cliente) > 0 and 'Name' in cliente[0]:
    nome_cliente = cliente[0]['Name']
    # Calcula o risco de churn
    risco_churn = ChurnCalculator.calcular_risco_churn(cases, data_primeiro_contrato, media_faturas)
else:
    risco_churn = 0


chat = model.start_chat(history=[])

# Imprime o risco de churn
#print(f"Risco de churn para o cliente {cpf}: {risco_churn:.2f}")

# Interação com o modelo de linguagem com base no risco de churn
if risco_churn > 0:
    prompt_churn_risk = f"O cliente {nome_cliente} dado falhas de nossos produtos ou atendimento esta em risco de churn. Seja criativo em uma resposta ou promoção de produtos de linha móvel para uma saudação e rete-lo, em nome da Empresa Fale Bem Telefonia."
    response = chat.send_message(prompt_churn_risk)
    print("Resposta do modelo:", response.text, "\n")
    prompt = input("Digite algo: ")
else:
    #print("O cliente não apresenta risco de churn.")
    prompt = input("Digite algo: ")


# Loop de conversa com o usuário
while prompt.lower() != "fim":
    response = chat.send_message(prompt)
    print("Resposta do modelo:", response.text, "\n")
    prompt = input("Digite algo: ")

print("Fim da conversa.")

Digite seu CPF: 972.237.264-58
Resposta do modelo: **Saudação Personalizada:**

Prezado Happy User,

Agradecemos profundamente por compartilhar suas valiosas opiniões conosco. Entendemos que você tenha enfrentado alguns desafios com nossos produtos ou atendimento, e levamos suas preocupações muito a sério.

**Reconhecimento e Empatia:**

Sabemos que as falhas podem ser frustrantes e queremos garantir que estamos comprometidos em melhorar sua experiência. Valorizamos sua fidelidade e queremos fazer tudo o que pudermos para mantê-lo como um cliente satisfeito.

**Promoção de Produtos de Linha Móvel:**

Para demonstrar nosso compromisso, gostaríamos de oferecer uma promoção exclusiva em nossos produtos de linha móvel de última geração. Esses dispositivos oferecem recursos avançados, conectividade confiável e atendimento ao cliente excepcional.

**Benefícios da Linha Móvel:**

* Conexão 5G ultrarrápida para streaming, jogos e downloads sem interrupções
* Câmeras de alta resolução para capt